In [1]:
import numpy as np

# Node Embeddings

<b> Recap </b>

Previously we discussed traditional feature based methods for graph ML, where given a graph G = (V,E) we wanted to extract node, link and graph-level features and learn a model (svm, logistic regressor, etc.) that maps features to labels.

Input Graph -> Feature Engineering -> Learning Algorithm -> Prediction

<b> Now </b>

With <b>Representation Learning</b> we automatically learn the features, without needing to manually construct the features as we did before (feature engineering). 

<b> Goal </b>

Efficient task-independent feature learning for machine learning with graphs

## Why Embeddings?

<b>Task:</b> map nodes into an embedding space

* Similarity of embeddings between nodes indicates their similarity in the network.
* Encode network information
* Potentially used for many downstream prediction tasks such as node classification, link prediction, graph classification, anomalous node detection, clustering, etc.

## Embedding Nodes

<b>Goal:</b> Find an encoder function such that, when nodes u and v are encoded to the embedding space, the following assumption is true:
the similarity between nodes u and v in the network is close to the similarity of the embedding representation of u and v in embedding space.

similarity(u,v) ~ z_v.T @ z_u

1. Encoder maps nodes to embeddings: Encoder(u) -> z_u.
2. Define a similarity function for the original network.
3. Decoder maps from embeddings to the embedding similarity score: dot preduct.
4. Optimize the parameters such that similarity(u,v) ~ decoder(encoder(u), encoder(v)).

This is an <b>unsupervised</b> way of learning node embeddings so we do not use labels or features. Also, the embeddings are task independent, because they are not trained for a specific task.

## "Shallow" Encoding

Simplest encoding approach: encoder is just an embedding-lookup. Each node is assigned a unique embedding vector, so we optimize the embedding of each node using one of many methods: DeepWalk, node2vec. Each optimization method defines its own similarity measure.

Encoder(v) = z_v = Z . v

Z in R ^ (d x |V|) - Z is a matrix with each column represents a node embedding (what we learn/optimize).

v in I ^ |V| - v is an indicator vector, all zeroes except a onde in column indicating node Vb

In [2]:
# 100 nodes with embedding size of 128
Z = np.random.randn(128, 100) #  this needs to be optimized somehow. In this example its a random matrix

# 100 entries for the indicator vetor, one for each node
v = np.zeros(100)
v[0] = 1

z = Z @ v

print("Dimensions -- Z:", Z.shape, "v:", v.shape, "z:", z.shape)
print("z is equal to the first column of Z:", (Z @ v == Z[:,0]).all())

Dimensions -- Z: (128, 100) v: (100,) z: (128,)
z is equal to the first column of Z: True


# Random-Walk Approaches for Node Embeddings

<b>Random Walk</b>

Given a graph G = (V, E) and a starting point v in V, we select a neighbor u in N(v) at random and move to u. Then we repeat this process for a random node z in N(u) and so on, creating a random sequence of nodes visited from a starting point.

Note: the same node can be visited multiple times in a random walk.

Provides:

1. Expressivity: flexible stochastic defintion of node similarity that incorporates both local and higher-order neighborhood information. Intuition: if random walk starting from node u visits v with high probability, u and v are similar (high-order multi-hop information).
2. Efficiency: do not need to consider all node pairs when training. Only need to consider pairs that co-occur on random walks. 

<b>Random-Walk Embeddings</b>

Goal: learn embeddings Z such that z_u.T @ z_v ~ probability that u and v co-occur on a random walk over the graph. In other words, nodes that appear togheter frequently in many random walks should have similar embedding representations.

1. Estimate the probability of visiting node v on a random walk starting from node u: P(v|u)
2. Optimize embeddings to encode these random walk statistics: cosine theta between z_u and z_v is proportional to P(v|u). This step is described in details below.

<b>Random Walk Optimization</b>

1. Run short fixed-length random walks starting from each node u in the graph.
2. For each node u collect N_r(u), the multiset of nodes visited on random walks starting from u. N_r(u) can repeat elements since nodes can be visited multiple times on random walks.
3. Optimize embeddings according to: given node u, predict its neighbors N_r(u) using z_u.

max (for each u in V -> log P(N_r(u)|z_u)) -- maximum likelihood objective.

Equivalently, we can write:

> L = for each u in V and for each v in N_r(u) -> -log(P(v|z_u))

Optimize embeddings z_u to maximize the likelihood of random walk co-occurrences.

P(v|z_u) is the softmax of the dot product between z_u and z_v. P(v|z_u) = softmax(z_u.T @ z_v). \*

Finally: find embeddings z_u that minimize L by applying stochastic gradient descent:

* initilize z_u randomly for each u in V.
* Iterate until converge:
* For all nodes u, compute the derivative of z_u w.r.t L.
* for all nodes u, make a step towards the diretion of the derivative: z_u <- z_u - learning_rate * derivative of z_u w.r.t L.

<b> * Consideration: Negative Sampling </b>

Softmax is defined as exp(z_u.T @ z_v)/(sum for each n in V -> exp(z_u.T @ z_n)). This is O(|V|^2) because of the denominator that iterates over every node in V. To reduce this computation, we use negative sampling.

Negative sampling is a technique that samples k nodes at random (usually k = 5 up to k = 20 ) instead of using all nodes in V. The k nodes are not sellected with uniform probability, but with a biased probability. The highest the degree of a node, the bigger the probability of it being sampled.

softmax(z_u.T @ z_v) ~ log(sigmoid(z_u.T @ z_v)) - sum from i to k -> log (sigmoid(z_u.T @ z_i))

